In [ ]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib import colors
pd.options.display.max_columns = None

In [ ]:
# Lectura y limpieza de datos
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Burak Yilmaz'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_m']=df['goals']/df['minutes_played'] # Cantidad de goles por minutos jugados
df['m_g']=df['minutes_played']/df['goals'] # Cantidad de minutos para que marque un gol
df['g_p']=df['goals']/df['match_played'] # Cantidad de goles por partidos jugados
df['a_m']=df['assists']/df['minutes_played'] # Cantidad de asistencias por minutos jugados
df['a_p']=df['assists']/df['match_played'] # Cantidad de asistencias por partidos jugados
df.replace([np.inf, -np.inf], np.nan, inplace=True) # eliminar infinitos
df.insert(36,'no_penalties',df['goals']-df['penalties'])
df.fillna(0,inplace=True)
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club')
equipos=pd.merge(equipos,df.groupby(['club'])['goals'].sum(), on='club')
equipos['g_m']=equipos['goals']/equipos['teams_matches']
df=pd.merge(df,equipos['teams_matches'],on='club')

In [ ]:
# Ver DataFrames
# df.columns
# df
# df.describe()
# equipos.columns
# equipos
# equipos.describe()

# Datos generales
Se presentan las estadisticas totales de la temporada

In [ ]:
# Obtener datos generales, se necesita para las siguientes celdas
totales=df['goals'].sum()
izquierda=round(df['left_foot'].sum()*100/totales,2)
derecha=round(df['right_foot'].sum()*100/totales,2)
cabeza=round(df['headers'].sum()*100/totales,2)
otros=round(df['others'].sum()*100/totales,2)
dentro_del_area=round(df['inside_area'].sum()*100/totales,2)
fuera_del_area=round(df['outside_areas'].sum()*100/totales,2)

In [ ]:
# Mostrar como texto los datos generales
print('Cantidad de equipos: {}'.format(len(pd.unique(df['club']))))
print('Cantidad de jugadores: {}'.format(len(pd.unique(df['player_name']))))
print("---------------------------------------------------")
print('Asistencias totales: {}'.format(int(df['assists'].sum())))
print('Goles totales: {}'.format(totales))
print("---------------------------------------------------")
print('Goles con la izquierda: {} --> %{}'.format(int(df['left_foot'].sum()),izquierda))
print('Goles con la derecha: {} --> %{}'.format(int(df['right_foot'].sum()),derecha))
print('Goles con la cabeza: {} --> %{}'.format(int(df['headers'].sum()),cabeza))
print('Otros: {} --> %{}'.format(int(df['others'].sum()),otros))
print("---------------------------------------------------")
print('Goles dentro del area: {} --> %{}'.format(int(df['inside_area'].sum()),dentro_del_area))
print('Goles fuera del area: {} --> %{}'.format(int(df['outside_areas'].sum()),fuera_del_area))
print("---------------------------------------------------")
print('Goles de penal: {}'.format(int(df['penalties'].sum())))
print("---------------------------------------------------")

In [ ]:
# Grafica de pastel, Tipos de goles
datos=[derecha,izquierda,cabeza,otros]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("jet")
colores =colormap(normdata(datos))
separacion=[0.02,0.02,0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Derecha','Izquierda','Cabeza','Otros'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Tipos de goles',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

In [ ]:
# Grafica de pastel, Goles dentro y fuera del area
datos=[dentro_del_area,fuera_del_area]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("turbo")
colores =colormap(normdata(datos))
separacion=[0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Dentro del area','Fuera del area'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Goles dentro y fuera del area',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

# Goleadores

In [ ]:
# Totales
df[['player_name','club','match_played','goals']].sort_values('goals',ascending=False).reset_index(drop=True).query("goals>0")

In [ ]:
# Por partido
df[['player_name','club','g_p']].sort_values('g_p',ascending=False).reset_index(drop=True).query("g_p>0")

In [ ]:
# Por minutos
df[['player_name','club','g_m']].sort_values('g_m',ascending=False).reset_index(drop=True).query("g_m>0")

In [ ]:
# Minutos por gol
df[['player_name','club','m_g']].sort_values('m_g').reset_index(drop=True).query("m_g>0")

In [ ]:
# Equipo total
df.groupby(['club'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Posicion
df.groupby(['position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

# Asistencias

In [ ]:
# Totales
df[['player_name','club','match_played','assists']].sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Por partido
df[['player_name','club','a_p']].sort_values('a_p',ascending=False).reset_index(drop=True).query("a_p>0")

In [ ]:
# Por minutos
df[['player_name','club','a_m']].sort_values('a_m',ascending=False).reset_index(drop=True).query("a_m>0")

In [ ]:
# Posicion
df.groupby(['position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

# Buscador

In [ ]:
# Buscador individual
bandera=True
while bandera or df_j.shape[0]==0:
    bandera=False
    jugador=input('Ingrese el nombre del jugador ')
    df_j=df[df['player_name']==jugador].reset_index(drop=True)
    if df_j.shape[0]>1:
        print(df_j[['player_name','club']])
        indice_jugador=int(input(f'Existen {df_j.shape[0]} coincidencias.\nIngresa el numero del jugadar que quieres seleccionar'))
        df_j=df_j.loc[indice_jugador]
if df_j['goals'][0]>0: # Grafica Distribucion de goles
    # Datos a graficar
    colores1=cm.get_cmap('Set1', 4).colors
    colores2=cm.get_cmap('Dark2', 2).colors
    colores3=cm.get_cmap('tab10', 2).colors
    x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
    x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
    x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
    plt.figure(figsize=(15,10)) # Crear grafica 1
    plt.subplot(111)
    plt.suptitle(f'Distribucion de goles de {jugador}',fontsize=21,fontweight='bold',color='w') # Titulo
    # Crear graficas
    x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    # Circulo central
    centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
    fig=plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Texto del circulo
    plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.text(-0.13,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.axis('equal') # Para que el circulo no tenga forma de elipse
    plt.legend(loc=7,fontsize=12) # Formato de la leyenda

    plt.show() # Mostrar
# Grafica Comparacion por posicion
# Datos a graficar
posicion=df_j['position'][0]
df_pos=df[df['position']==posicion]
fig,ax=plt.subplots(nrows=3, ncols=2, figsize=(25,10)) # Crear figura
plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion',fontsize=21,fontweight='bold',color='w') # Titulo
columnas=[['goals','assists'],['g_p','a_p'],['g_m','a_m']] # Columnas a graficar
# Graficar
for i,col in zip(range(3),columnas):
    # Graficas izquierda
    df_pos.boxplot([col[0]], ax=ax[i][0],vert=False)
    ax[i][0].plot(df_j[col[0]][0],1, marker="D", color="red",markersize=9)
    ax[i][0].tick_params(labelsize=15,labelcolor='w')
    ax[i][0].set_facecolor('azure')
    # Graficas derecha
    df_pos.boxplot([col[1]], ax=ax[i][1],vert=False)
    ax[i][1].plot(df_j[col[1]][0],1, marker="D", color="red",markersize=9)
    ax[i][1].tick_params(labelsize=15,labelcolor='w')
    ax[i][1].set_facecolor('azure')
plt.show()
plt.tight_layout()